In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [53]:
lista_ligas = [
    "https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats",
    "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats",
    "https://fbref.com/en/comps/11/2021-2022/2021-2022-Serie-A-Stats",
    "https://fbref.com/en/comps/13/2021-2022/2021-2022-Ligue-1-Stats",
    "https://fbref.com/en/comps/20/2021-2022/2021-2022-Bundesliga-Stats",
]

years = list(range(2021, 2013, -1))
all_matches = []
log_error = []
log = []
for year in years:
    for indice, liga in enumerate(lista_ligas):
        data = requests.get(liga)
        soup = BeautifulSoup(data.text)

        try:
            standings_table = soup.select("table.stats_table")[0]
        except Exception as e:
            
            continue

        links = [l.get("href") for l in standings_table.find_all("a")]
        links = [l for l in links if "/squads/" in l]
        team_urls = [f"https://fbref.com{l}" for l in links]

        previous_season = soup.select("a.prev")[0].get("href")
        

        time.sleep(1)
        for team_url in team_urls:
            
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            data = requests.get(team_url)
            time.sleep(1)
            try:
                matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
                matches = matches[
                    [
                        "Date",
                        "Time",
                        "Comp",
                        "Round",
                        "Venue",
                        "Result",
                        "GF",
                        "GA",
                        "Opponent",
                        "Poss",
                        "Formation",
                    ]
                ]
                soup = BeautifulSoup(data.text)
                links = [l.get("href") for l in soup.find_all("a")]
                time.sleep(1)
            except Exception as e:
                log_error.append(f"error:{e},{lista_ligas[indice]},{team_url}")
                continue

            try:
                links_shooting = [l for l in links if l and "all_comps/shooting/" in l]
                data_shooting = requests.get(f"https://fbref.com{links_shooting[0]}")
                shooting = pd.read_html(data_shooting.text, match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()
                time.sleep(1)
            except Exception as e:
                
                continue

            try:
                links_misc = [l for l in links if l and "all_comps/misc/" in l]
                data_misc = requests.get(f"https://fbref.com{links_misc[0]}")
                defensive = pd.read_html(data_misc.text, match="Miscellaneous Stats")[0]
                defensive.columns = defensive.columns.droplevel()
                time.sleep(1)

            except Exception as e:
                continue

            try:
                team_data = matches.merge(defensive[["Date", "TklW", "Int"]], on="Date")
            except ValueError:
                continue

            try: 
                if 'xG' in shooting.columns:
                    team_data = team_data.merge(
                    shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt", "xG"]],
                        on="Date",
                    )
                    print('entre en el if')
                    print(shooting.columns)
                else:
                    shooting["xG"] = ''
                    team_data = team_data.merge(
                    shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt", "xG"]], on="Date"
                )
                    print('entre en el else')
                    print(shooting.columns)

            except Exception as e:
              
                continue

            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
            time.sleep(2)
            pass

        lista_ligas[indice] = f"https://fbref.com{previous_season}"
     


entre en el if
Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Gls', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK',
       'PK', 'PKatt', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG',
       'Match Report'],
      dtype='object')
entre en el if
Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Gls', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK',
       'PK', 'PKatt', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG',
       'Match Report'],
      dtype='object')
entre en el if
Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Gls', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK',
       'PK', 'PKatt', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG',
       'Match Report'],
      dtype='object')
entre en el if
Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Gls', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/

In [54]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df = match_df[['date', 'time', 'season', 'team', 'comp', 'round', 'venue', 'result', 'gf', 'ga',
       'opponent', 'poss', 'formation', 'tklw', 'int', 'sh', 'sot', 'dist',
       'pk', 'pkatt', 'xg']]
match_df.to_csv('matches.csv')

In [55]:
match_df.tail()

,date,time,season,team,comp,round,venue,result,gf,ga,...,poss,formation,tklw,int,sh,sot,dist,pk,pkatt,xg
40,2021-04-25,15:00,2020,Rennes,Ligue 1,Matchweek 34,Home,W,5,1,...,74.0,4-3-3,9.0,6.0,25.0,11.0,16.7,0.0,0.0,3.0
41,2021-05-02,13:00,2020,Rennes,Ligue 1,Matchweek 35,Away,L,0,1,...,55.0,4-3-3,13.0,9.0,9.0,2.0,19.0,0.0,0.0,0.6
42,2021-05-09,21:00,2020,Rennes,Ligue 1,Matchweek 36,Home,D,1,1,...,43.0,4-4-2,7.0,10.0,19.0,10.0,18.9,0.0,0.0,1.0
43,2021-05-16,21:00,2020,Rennes,Ligue 1,Matchweek 37,Away,L,1,2,...,60.0,4-3-3,5.0,5.0,9.0,2.0,20.5,0.0,0.0,0.4
44,2021-05-23,21:00,2020,Rennes,Ligue 1,Matchweek 38,Home,W,2,0,...,68.0,4-4-2,7.0,10.0,12.0,6.0,19.8,0.0,0.0,1.2


In [57]:
all_matches[-1]

,Date,Time,Comp,Round,Venue,Result,GF,GA,Opponent,Poss,...,TklW,Int,Sh,SoT,Dist,PK,PKatt,xG,Season,Team
0,2020-08-22,21:00,Ligue 1,Matchweek 1,Away,D,1,1,Lille,48.0,...,9,11,16,2,17.7,0,0,1.6,2020,Rennes
1,2020-08-29,17:00,Ligue 1,Matchweek 2,Home,W,2,1,Montpellier,65.0,...,15,8,16,7,21.3,0,0,1.4,2020,Rennes
2,2020-09-13,15:00,Ligue 1,Matchweek 3,Away,W,4,2,Nîmes,55.0,...,11,4,10,6,16.3,0,0,2.1,2020,Rennes
3,2020-09-19,21:00,Ligue 1,Matchweek 4,Home,W,2,1,Monaco,58.0,...,4,9,15,3,17.9,0,0,1.0,2020,Rennes
4,2020-09-26,17:00,Ligue 1,Matchweek 5,Away,W,3,0,Saint-Étienne,51.0,...,11,12,10,6,16.6,0,0,1.0,2020,Rennes
5,2020-10-04,17:00,Ligue 1,Matchweek 6,Home,D,2,2,Reims,73.0,...,5,7,19,3,16.6,0,0,1.8,2020,Rennes
6,2020-10-16,19:00,Ligue 1,Matchweek 7,Away,D,1,1,Dijon,60.0,...,15,14,12,3,19.8,0,0,0.9,2020,Rennes
7,2020-10-20,21:00,Champions Lg,Group stage,Home,D,1,1,ru Krasnodar,62.0,...,9,7,19,6,19.8,1,1,2.7,2020,Rennes
8,2020-10-23,21:00,Ligue 1,Matchweek 8,Home,L,1,2,Angers,63.0,...,9,10,11,1,14.5,0,0,1.4,2020,Rennes
9,2020-10-28,21:00,Champions Lg,Group stage,Away,L,0,1,es Sevilla,34.0,...,5,11,2,2,27.4,0,0,0.1,2020,Rennes
